# Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import geopy.distance as gp

# Carga dos Dados

In [ ]:
poi = pd.read_csv('../data/base_pois_def.csv')
position = pd.read_csv('../data/posicoes.csv')

In [ ]:
# Converte data_posicao para datetime
position['data_posicao'] = pd.to_datetime(position['data_posicao'], format="%a %b %d %Y %H:%M:%S GMT%z", exact=False)

# Determina se o veiculo estah parado (on nao) em cada registro
position['parado'] = position.apply(lambda x: (not x.ignicao and x.velocidade < 5), axis = 1)

# Calcula o tempo(s) entre dois registros consecutivos
position = position.sort_values(by=['placa','data_posicao'])
position['tempo'] = (position['data_posicao'] - position.groupby(['placa'])['data_posicao'].shift(1)).dt.seconds

# 'JOIN' dos dois DataFrames

In [ ]:
data = []
for aPos in position.itertuples():
    for aPoi in poi.itertuples():
        # Calcula a distancia do veiculo ao POI
        dist = gp.distance((aPos.latitude, aPos.longitude), (aPoi.latitude, aPoi.longitude)).meters
        # Cria um registro, se o veiculo estiver em um POI
        if (dist <= aPoi.raio):
            data.append({'placa': aPos.placa, 'tempo': aPos.tempo, 'poi': aPoi.nome, 'parado': aPos.parado})

positionPoi = pd.DataFrame(data)

# Consolidacao dos tempos

In [ ]:
result = positionPoi.groupby(['placa','poi','parado'], as_index=False)['tempo'].sum()
result

In [ ]:
# Quantidade de tempo que os veiculos passaram parados dentro de cada POI
carStoppedPoi = result[result['parado']==True].groupby(['placa','poi'], as_index=False)['tempo'].sum()
carStoppedPoi

In [ ]:
# Quantidade de tempo que os veiculos estavam dentro de cada POI
carOnPoi = result.groupby(['placa','poi'], as_index=False)['tempo'].sum()
carOnPoi

In [ ]:
# Tempo total da frota gasto parado em cada POI
fleetStoppedPoi = result[result['parado']==True].groupby(['poi'], as_index=False)['tempo'].sum()
fleetStoppedPoi

In [ ]:
# Tempo total parado por veiculo, independente do POI
carStopped = position[position['parado']==True].groupby(['placa'], as_index=False)['tempo'].sum()
carStopped

# Consolidacao dos dados

In [ ]:
data = []
car = np.unique(np.append(pd.unique(result['placa']), pd.unique(result['placa'])))
for aCar in car:
    st = {'placa': aCar}
    # Adiciona colunas com o tempo parado em cada POI
    for aRec in carStoppedPoi[carStoppedPoi['placa'] == aCar].itertuples():
        st[aRec.poi.replace(' ','_').lower()+'_parado'] = aRec.tempo

    # Adiciona colunas com o tempo em cada POI
    for aRec in carOnPoi[carOnPoi['placa'] == aCar].itertuples():
        st[aRec.poi.replace(' ','_').lower()+'_total'] = aRec.tempo
    
    # Adiciona coluna com o tempo total parado
    for aRec in carStopped[carStopped['placa'] == aCar].itertuples():
        st['total_parado'] = aRec.tempo
    
    data.append(st)

st = {'placa': 'FROTA'}
# Adiciona colunas com o tempo da frota parada em cada POI
for aRec in fleetStoppedPoi.itertuples():
    st[aRec.poi.replace(' ','_').lower()+'_parado'] = aRec.tempo
data.append(st)

output = pd.DataFrame(data).fillna(0)
output

# Output dos resultados

In [ ]:
cols = output.columns.tolist()
cols.sort()
output[cols].to_csv('../output/resultados_consolidados_POIs.csv', index=False)